### Phương pháp Đơn hình Dantzig (Trường hợp bi > 0)

In [1]:
import numpy as np
from sympy import*
import math

### Sử dụng thư viện Scipy

In [5]:
import numpy as np
from scipy.optimize import linprog

In [14]:
A = np.array([[0.5, -5.5, -2.5, 9], [0.5, -1.5, -0.5, 1], [1, 0, 0, 0]])
c = np.array([-10, 57, 9, 24])
b = np.array([0, 0 ,1])
bounds = [(0, None), (0, None), (0, None), (0, None)]
constraints = [{'type': 'ineq', 'fun': lambda x: b - A.dot(x)}]

In [15]:
res = linprog(c, A_ub=A, b_ub=b, method='simplex')

In [16]:
print(res)

     con: array([], dtype=float64)
     fun: 0.0
 message: 'Iteration limit reached.'
     nit: 1000
   slack: array([0., 0., 1.])
  status: 1
 success: False
       x: array([0., 0., 0., 0.])


### Sử dụng thư viện để kiểm tra

In [240]:
from scipy.optimize import linprog
import numpy as np
# Hàm mục tiêu và ràng buộc của bài toán tối ưu hóa
#c = [1, -2, 3]
#A = [[-1, 2, -3], [4, -5, 6], [-7, 8, -9]]
#b = [-1, 2, -3]
A = np.array([[-3,-1], [-1,-2], [1,-1], [1,0], [0,1]])
c = np.array([-1,1])
b = np.array([-3.,-4.,1.,5.,5.])


# Giải bài toán tối ưu hóa bằng phương pháp đơn hình Bland
res = linprog(c, A_ub=A, b_ub=b, method='simplex', options={'bland': True})

# In kết quả
print('Nghiệm tối ưu: ', res.x)
print('Giá trị hàm mục tiêu tối ưu: ', res.fun)

Nghiệm tối ưu:  [5. 4.]
Giá trị hàm mục tiêu tối ưu:  -1.0000000000000009


In [252]:
import numpy as np

# A: ma trận chưa hệ của của các biến trong các ràng buộc
# b: mảng chứa các giá trị bi
# c: mảng chứa hệ số các biến trong hàm mục tiêu

def dantzig_method(A, b, c):
    m, n = A.shape

    # Khởi tạo biến giá trị tối ưu
    z = 0 
    
    # Tính ma trận bổ sung
    T = np.hstack((A, np.eye(m)))
    c_n = np.concatenate((c, np.zeros(m)))
    b_n = b.copy()
    
    # Copy biến ban đầu để so sánh
    A_cop = T.copy()
    c_cop = c_n.copy()
    b_cop = b.copy()
    
    # Tính biến cơ sở ban đầu
    basis = list(range(n, n + m))
    # Chuyển về dạng chuẩn tắc
    while True:
        # Tìm biến vào
        j = np.argmin(c_n)
        
        if c_n[j] >= 0:
            break
            
        # Tìm biến ra
        ratios = [float('inf')] * m
        for i in range(m):
            if T[i, j] > 0:
                ratios[i] = b_n[i] / T[i, j]
        
        i = np.argmin(ratios)

        # Kiểm tra bài toán không giới nội
        if ratios[i] == float('inf'):
            print("Bài toán không giới nội.")
            return -np.inf, None
        
        # Tính giá trị tối ưu
        z += c_n[j] / T[i, j] * b_n[i]
        # Cập nhật ma trận
        basis[i] = j
        
        for k in range(m):
            if k != i:
                b_n[k] -= T[k, j]/T[i,j] * b_n[i]
                T[k, :] -=  T[k, j] / T[i,j] * T[i, :]
        
        b_n[i] /= T[i, j]
        T[i, :] /= T[i, j]       
        c_n -= c_n[j] * T[i, :]
        
        # Kiểm tra thuật toán có hoàn thành hay không 
        # Nếu Từ vựng mới trùng với từ vựng ban đầu thì dừng thuật toán
        if (c_n == c_cop).all() and (T == A_cop).all() and (b_n == b_cop).all():
            print("Thuật toán lặp vô hạn.")
            return None, None
        
    # Tìm nghiệm tối ưu
    x = np.zeros(n)
    for i in range(m):
        if basis[i] < n:
            x[basis[i]] = b_n[i]
                           
    return z, x 

#### bi >0

In [253]:
c = np.array([-3,-1, 0])
A = np.array([[1,2,0], [1,1,-1], [7,3,-5]])
b = np.array([5.,2.,20.])
z, x = dantzig_method(A, b, c)
print("z = ", z)
print("x = ", x)

z =  -15.0
x =  [5. 0. 3.]


In [254]:
c = np.array([-4,-5])
A = np.array([[2,2], [1,0], [0,1]])
b = np.array([9.,4.,3.])
z, x = dantzig_method(A, b, c)
print("z = ", z)
print("x = ", x)

z =  -21.0
x =  [1.5 3. ]


#### Trường hợp không giới nội

In [255]:
c = np.array([-1,-3,1])
A = np.array([[2,2,-1],[3,-2,1], [1,-3,1]])
b = np.array([10.,10., 10.])
#c = np.array([-4, -5, 5])
#A = np.array([[1,2,-2], [1,-1, 1], [2,1,-1], [3,4,-4]])
#b = np.array([3,2,3,8])
z, x = dantzig_method(A, b, c)
print("z = ", z)
print("x = ", x)

Bài toán không giới nội.
z =  -inf
x =  None


#### Trường hợp vô số nghiệm

In [258]:
c = np.array([-1, 1])
A = np.array([[-3,-1], [-1,-2], [1,-1], [1,0], [0,1]])
b = np.array([-3., -4., 1., 5., 5.])
z, x = dantzig_method(A, b, c)
print("z = ", z)
print("x = ", x)

z =  -1.0
x =  [1. 0.]


#### Trường hợp lặp vô hạn

In [239]:
A = np.array([[0.5, -5.5, -2.5, 9], [0.5, -1.5, -0.5, 1], [1, 0,0,0]])
c = np.array([-10, 57, 9, 24])
b = np.array([0.,0.,1.])
z, x = dantzig_method(A, b, c)
print("z = ", z)
print("x = ", x)

ratios: [0.0, 0.0, 1.0]
ratios: [inf, 0.0, 0.09090909090909091]
ratios: [0.0, 0.0, inf]
ratios: [inf, 0.0, inf]
ratios: [0.0, 0.0, inf]
ratios: [inf, 0.0, inf]
Thuat toan lap vo han.
z =  None
x =  None


In [248]:
import numpy as np

A = np.array([[-1,0,1], [0,3,-3], [1,1,-1], [4,-1,1], [5, -1, 0]])
Q, R = np.linalg.qr(A)

if Q.shape[1] == A.shape[1]:
    print("Ma trận A khả giải.")
else:
    print("Ma trận A không khả giải.")

Ma trận A khả giải.
